# 9. 문서 요약 (Text Summarization)

## 9-0 데이터 준비

In [50]:
import requests 
from bs4 import BeautifulSoup
import re

def cleansing(soup):
    body_content = soup.select_one("#articleBodyContents")
    content = body_content.get_text()
    for strong in body_content.select("strong"):
        content = content.replace(strong.get_text(), "")

    for td in body_content.select("td"):
        content = content.replace(td.get_text(), "")
    
    content = re.sub(r"\[[가-힣 ]+\]", "", content)
    start_pos = re.search(r"[가-힣]{2,4}\s\(?\w+@\w+\.\w+(.\w+)?\)", content).start()
    content = content[:start_pos-1]
    content = content.replace("\n", "").replace("// flash 오류를 우회하기 위한 함수 추가function _flash_removeCallback() {}", "")


    return content

def get_news_by_url(url):
    headers={"user-agent":"Mozilla/5.0"}
    res = requests.get(url, headers=headers)
    soup = BeautifulSoup(res.content, "html.parser")
    content = cleansing(soup)
    return content

doc = get_news_by_url('https://news.naver.com/main/read.nhn?mode=LSD&mid=sec&sid1=105&oid=018&aid=0004430108')
print(doc[:50])
print(doc[-50:])


 국가 차원의 빅데이터 활용 시대가 열린다. 새로운 산업 창출과 기존 산업의 변화에 이르는
를 혁신하고 기업의 경쟁력을 한 단계 제고할 수 있도록 정책적 역량을 집중하겠다”고 밝혔다


## 9-1 Luhn Summarizer

In [51]:
!pip install kss

1) 토큰화

In [52]:
from nltk.tokenize import sent_tokenize
from kss import split_sentences
from konlpy.tag import Mecab

mecab = Mecab()

# 문장 분리
def get_sentences(txt):
    return split_sentences(txt)
#     return sent_tokenize(txt)
# 토큰화
def get_words(txt):
    return [token[0] for token in mecab.pos(txt) if token[1][0] == "N" and len(token[0]) >1]



2) 중요 단어 결정

In [53]:
# 단어(토큰)의 가중치 계산 및 범위에 포함되는 토큰 식별 
def get_keywords(word_list , min_ratio=0.001, max_ratio=0.5) :
    assert (min_ratio < 1 and max_ratio < 1)
    
    # 토큰별로 빈도수 카운팅
    count_dict = {}
        
    for word in word_list :
#         if word in count_dict.keys():
#             count_dict[word] += 1
#         else :
#             count_dict[word] = 1
        count_dict.setdefault(word, 0)
        count_dict[word] += 1
    
# 분석 문서의 총 토큰수 대비 해당 토큰의 빈도 비율
    keywords = set()
    for word, cnt in count_dict.items():
        word_percentage = cnt / len(word_list)
        
         # 사전 정의한 비율내에 포함 된 경우 키워드에 추가
        if word_percentage >=min_ratio and word_percentage <= max_ratio:
            keywords.add(word)

    return keywords

get_keywords(['바나나','사과','바나나','바나나','포도'])

{'사과', '포도'}

3) 문장 중요도 계산

In [54]:
# 문장의 가중치 계산
def get_sentence_weight (sentence , keywords):
    tokens = sentence.split(" ") # 문장을 받고 리스트가 되고
    window_start, window_end = 0, -1
    
    # 문장내에서 윈도 시작 위치 탐색
    # 범위내 속한 키워드가 등장하는 첫번째 위치 계산
    for i in range(len(tokens)):
        if sentence[i] in keywords: # keywords는 set() 값!
            window_start = i
            break
    
    # 문장내에서 윈도 종료 위치 탐색
    # 범위내 속한 키워드가 등장하는 마지막 위치 계산
    for i in range(len(tokens)-1, 0, -1):
        if tokens[i] in keywords:
            window_end = i 
            break
    
    # 윈도의 시작위치가 종료위치보다 큰경우 => 분석할 단어(토큰)가 없는 경우 종료
    if window_start > window_end:
        return 0
    
       # 윈도 크기 계산
    window_size = window_end - window_start + 1

    
    # 분석 대상 문장 중 범위(0.001 ~ 0.5)에 포함된 토큰 개수 카운팅
    keyword_cnt = 0
    for w in tokens[window_start : window_start+ window_size]:
        if w in keywords:
            keyword_cnt += 1
    
    # (분석 대상 문장 중 범위(0.001 ~ 0.5)에 포함된 토큰 개수) / 윈도사이즈
    return  keyword_cnt * keyword_cnt * 1.0 / float(window_size)

4) 문서 요약

In [55]:
# 문서 요약
def summarize(content ,max_no_of_sentences = 10):
    
    
    # 단어(토큰) 분리
    word_list = get_words(content)
    
    # 단어(토큰) 가중치 계산 및 범위 내 포함 단어(토큰) 추출
    keywords = get_keywords(word_list)

    # 문장별 가중치 계산
    sentence_list = get_sentences(content)
    sentence_weight = []
    
    for sentence in sentence_list :
        sentence_weight.append((get_sentence_weight(sentence, keywords), sentence))
        
           
    # 문장별 가중치 역순 계산
    sentence_weight.sort(reverse=True)
#     print(sentence_weight)
    
    
    return [ weight[1] for weight in sentence_weight[:max_no_of_sentences]]

In [56]:
li = summarize(doc, 3)
for s in li:
    print(s)
    print("\n\n\n")

22일 과학기술정보통신부는 서울 중구 대한상공회의소에서 데이터 생태계 조성과 혁신 성장의 기반 마련을 위한 ‘빅데이터 플랫폼 및 센터’ 출범식 행사를 개최했다.




금융 플랫폼의 경우 소상공인 신용평가 고도화 등을 통해 금융 취약 계층 대상 중금리 대출이자를 2%p 절감해 연간 1조원의 신규대출을 창출할 전망이다.




빅데이터는 데이터 활용을 통해 혁신성장을 이루자는 문재인 정부의 경제 성장 핵심 요소중 하나다.






## 2-1 Text rank 직접 구현하기 (Matrix 활용)

1) 자카드 유사도

In [86]:
Text = "딸기 바나나 사과 파인애플. 바나나 사과 딸기 포도. 복숭아 수박. 파인애플 사과 딸기 바나나."

In [111]:
from konlpy.tag import Mecab
mecab = Mecab()

# 문장간 유사도 측정 (자카드 유사도 사용)
def sentence_similarity(sentence1, sentence2):
    sentence1 = [token[0] for token in mecab.pos(sentence1) if token[1][0] in ["N", "V"]]
    sentence2 = [token[0] for token in mecab.pos(sentence2) if token[1][0] in ["N", "V"]]
    
    union = set(sentence1).union(set(sentence2))
    intersection = set(sentence1).intersection(set(sentence2))
    
    return len(intersection)/len(union)
  

sentence_similarity('나는 치킨을 좋아해','나는 치킨을 싫어해')

0.5

2) 그래프 생성

In [118]:
for token in mecab.pos("나는 치킨을 좋아해") :
    if token[1][0] in ["N", "V"]:
        print(token[0])


나
치킨
좋아해


In [88]:
import numpy as np
def buildMatrix(sentences):
    score = np.ones(len(sentences), dtype=np.float32)
    weighted_edge = np.zeros((len(sentences), len(sentences)), dtype=np.float32)

    # 문장별로 그래프 edge를 Matrix 형태로 생성
    for i in range(len(sentences)):
        for j in range(len(sentences)):
            if i == j:
                continue

            weighted_edge[i][j] = sentence_similarity(sentences[i], sentences[j])

    # normalize 
    for i in range(len(weighted_edge)):
        score[i] = weighted_edge[i].sum()
        weighted_edge[i] /= score[i]
    
    return  weighted_edge


Text = "딸기 바나나 사과 파인애플 수박. 바나나 사과 딸기 포도. 복숭아 수박. 파인애플 사과 딸기 바나나."
buildMatrix(sent_tokenize(Text))    

array([[0.        , 0.3409091 , 0.11363637, 0.54545456],
       [0.45454544, 0.        , 0.        , 0.54545456],
       [1.        , 0.        , 0.        , 0.        ],
       [0.57142854, 0.4285714 , 0.        , 0.        ]], dtype=float32)

3) 문장 중요도 계산

In [89]:
def scoring(weight_edge, score, threshold=0.0001, d=0.85, max_iter = 50):
    for iter in range(max_iter):
        new_score = (1-d) + d *  weight_edge.T.dot(score)

        if abs(new_score - score).sum() <= threshold:
            break
    
        score = new_score
    
    return new_score
        

4) 문서 요약

In [106]:
import nltk
from nltk.tokenize import sent_tokenize

def summarize(text, n=10):
    
#   sentences = sent_tokenize(text)
    sentences = get_sentences(text)
  
    weighted_edge = buildMatrix(sentences)
    init_score = np.ones(len(sentences), dtype=np.float32)
    score = scoring(weighted_edge, init_score)

    sorted_score = sorted(enumerate(score), key=lambda x : x[1], reverse=True)[:n]
    return [ sentences[sent[0]] for sent in sorted_score ]

In [105]:
summary = summarize(Text, 3)

for sent in summary :
    print(sent)

('딸기 바나나 사과 파인애플.', 0.3430375429643529)
('파인애플 사과 딸기 바나나.', 0.3430375429643529)
('바나나 사과 딸기 포도.', 0.2663058284544005)


In [95]:
summary = summarize(doc, 3)

for sent in summary :
    print(sent)

이런 맥락 속에서 빅데이터센터는 공공과 민간이 협업해 활용도 높은 양질의 데이터를 생산·구축하고, 플랫폼은 이를 수집·분석·유통하는 역할을 담당한다.
100개 센터에서 수집된 데이터를 융합·분석한 뒤 맞춤형 데이터 제작 등 양질의 데이터로 재생산하고, 기업들이 필요로 하는 데이터를 원하는 형태로 즉시 활용할 수 있도록 제공할 계획이다.
센터와 플랫폼 간 연계체계에는 민간 클라우드를 기반으로 활용하고, 센터에 축적된 데이터도 계속 외부와 개방·공유하며 최신·연속성을 확보한다는 계획이다.


## 2-2 TextRank 직접 구현하기 (Graph 활용)

In [96]:
Text = "딸기 바나나 사과 파인애플. 바나나 사과 딸기 포도. 복숭아 수박. 파인애플 사과 딸기 바나나."

1) 토큰화

In [97]:
import nltk
nltk.download('punkt')
from nltk.tokenize import sent_tokenize

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\rnru1\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [98]:
def sentences(text):
    return sent_tokenize(text)

2) 자카드 유사도

In [99]:
from konlpy.tag import Mecab
mecab = Mecab()

# 문장간 유사도 측정 (자카드 유사도 사용)
def sentence_similarity(sentence1, sentence2):
    sentence1 = [token[0] for token in mecab.pos(sentence1) if token[1][0] in ["N", "V"]]
    sentence2 = [token[0] for token in mecab.pos(sentence2) if token[1][0] in ["N", "V"]]
    
    union = set(sentence1).union(set(sentence2))
    intersection = set(sentence1).intersection(set(sentence2))
    
    return len(intersection)/len(union)
  

sentence_similarity('나는 치킨을 좋아해','나는 치킨을 싫어해')

0.5

In [103]:
def connect(nodes):
    return [(start, end, sentence_similarity(start, end)) for start in nodes for end in nodes if start is not end] 

In [121]:
nodes = ["딸기 바나나 사과 파인애플", "바나나 사과 딸기 포도", "복숭아 수박", "파인애플 사과 딸기 바나나"]
for start in nodes :
    for end in nodes :
        if start is not end:
            print(start, end)

딸기 바나나 사과 파인애플 바나나 사과 딸기 포도
딸기 바나나 사과 파인애플 복숭아 수박
딸기 바나나 사과 파인애플 파인애플 사과 딸기 바나나
바나나 사과 딸기 포도 딸기 바나나 사과 파인애플
바나나 사과 딸기 포도 복숭아 수박
바나나 사과 딸기 포도 파인애플 사과 딸기 바나나
복숭아 수박 딸기 바나나 사과 파인애플
복숭아 수박 바나나 사과 딸기 포도
복숭아 수박 파인애플 사과 딸기 바나나
파인애플 사과 딸기 바나나 딸기 바나나 사과 파인애플
파인애플 사과 딸기 바나나 바나나 사과 딸기 포도
파인애플 사과 딸기 바나나 복숭아 수박


3) 그래프 생성

In [100]:
import networkx as nx

def rank(nodes,edges):
    graph=nx.diamond_graph()
    graph.clear() 
    graph.add_nodes_from(nodes)
    graph.add_weighted_edges_from(edges)

    return nx.pagerank(graph)

4) 문서 요약

In [101]:
def summarize(text, num_summaries=3):
    nodes = sentences(text)
    edges = connect(nodes)
    scores = rank(nodes, edges)
    return sorted(scores.items(), key=lambda x : x[1], reverse=True)[:num_summaries]

In [104]:
summary=summarize(Text, 3)

for sent in summary :
    print(sent)

('딸기 바나나 사과 파인애플.', 0.3430375429643529)
('파인애플 사과 딸기 바나나.', 0.3430375429643529)
('바나나 사과 딸기 포도.', 0.2663058284544005)
